# CMR Resource Configurator

In [1]:
from setvar import *

In [2]:
import re, os, sys

In [3]:
setvar("PATH=/usr/local/cli/bin:/usr/bin:/bin:/usr/sbin:/usr/local/sbin:/sbin")

PATH=/usr/local/cli/bin:/usr/bin:/bin:/usr/sbin:/usr/local/sbin:/sbin


## Step 1: Select your tenant

In [4]:
!tenants-list

ID                   NAME                                     URL                                               
araport.org          Araport                                  https://api.araport.org/                          
agave.prod           Agave Public Tenant                      https://public.agaveapi.co/                       
portals              Portals Tenant                           https://portals-api.tacc.utexas.edu/              
sgci                 Science Gateways Community Institute     https://sgci.tacc.cloud/                          
bridge               Bridge                                   https://api.bridge.tacc.cloud/                    
designsafe           DesignSafe                               https://agave.designsafe-ci.org/                  
sd2e                 SD2E Tenant                              https://api.sd2e.org/                             
tacc.prod            TACC                                     https://api.tacc.utexas.edu/      

In [5]:
#!tenants-init -t sandbox

## Step 2: Set your Agave Username and Password

In [6]:
setvar("AGAVE_USERNAME=ysboss")

AGAVE_USERNAME=ysboss


In [7]:
#os.remove("AGAVE_PASSWD.txt")
readpass("AGAVE_PASSWD")

Password or secret: AGAVE_PASSWD
Reading file `AGAVE_PASSWD.txt'


## Step 3: Provide Information about the Resource
Here you should supply the fully qualified domain name
of the machine, the port it will be accessed on, the
user that will be accessing the machine, etc.

In [8]:
setvar("""
MACHINE_USERNAME=funwave
MACHINE_FULL=shelob.hpc.lsu.edu
PORT=22
ALLOCATION=hpc_cmr
BASE_APP_NAME=crcollaboratory
WORK_DIR=/work/${MACHINE_USERNAME}
HOME_DIR=/home/${MACHINE_USERNAME}
SCRATCH_DIR=/work/${MACHINE_USERNAME}
DEPLOYMENT_PATH=agave-deployment
AGAVE_JSON_PARSER=jq
""")

MACHINE_USERNAME=funwave
MACHINE_FULL=shelob.hpc.lsu.edu
PORT=22
ALLOCATION=hpc_cmr
BASE_APP_NAME=crcollaboratory
WORK_DIR=/work/funwave
HOME_DIR=/home/funwave
SCRATCH_DIR=/work/funwave
DEPLOYMENT_PATH=agave-deployment
AGAVE_JSON_PARSER=jq


## Step 4: Provide the login credentials

In [9]:
#os.remove("MACHINE_PASSWD.txt")
readpass("MACHINE_PASSWD")

Password or secret: MACHINE_PASSWD
Reading file `MACHINE_PASSWD.txt'


In [10]:
# Parse out the name of the machine from its domain
g = re.match(r'(\w+)\.(.*)',os.environ["MACHINE_FULL"])
setvar("""
MACHINE={}
DOMAIN={}
""".format(g.group(1),g.group(2)))

MACHINE=shelob
DOMAIN=hpc.lsu.edu


In [11]:
setvar("""
BASE_APP_NAME2=${BASE_APP_NAME}-tapis-${MACHINE}-${AGAVE_USERNAME}
STORAGE_MACHINE=${MACHINE}-tapis-storage-${AGAVE_USERNAME}
EXEC_MACHINE=${MACHINE}-tapis-exec-${AGAVE_USERNAME}
""")

BASE_APP_NAME2=crcollaboratory-tapis-shelob-ysboss
STORAGE_MACHINE=shelob-tapis-storage-ysboss
EXEC_MACHINE=shelob-tapis-exec-ysboss


In [12]:
#!clients-delete -u $AGAVE_USERNAME -p $AGAVE_PASSWD $BASE_APP_NAME2
#!clients-create -p $AGAVE_PASSWD -S -N $BASE_APP_NAME2 -u $AGAVE_USERNAME
#!auth-tokens-create -u $AGAVE_USERNAME -p $AGAVE_PASSWD
!auth-session-init -t tacc.prod -u $AGAVE_USERNAME -p $AGAVE_PASSWD -N $BASE_APP_NAME2

Loading crcollaboratory-tapis-shelob-ysboss for ysboss from tacc.prod
Refreshing access token for crcollaboratory-tapis-shelob-ysboss...
129abda9aeb7b9e49a66fc899187a37


In [13]:
writefile("${STORAGE_MACHINE}.txt","""{
    "id": "${STORAGE_MACHINE}",
    "name": "${MACHINE} storage (${MACHINE_USERNAME})",
    "description": "The ${MACHINE} computer",
    "site": "${DOMAIN}",
    "type": "STORAGE",
    "storage": {
       "host": "${MACHINE_FULL}",
       "port": ${PORT},
       "protocol": "SFTP",
       "rootDir": "/",
       "homeDir": "${HOME_DIR}",
       "auth": {
           "username" : "${MACHINE_USERNAME}",
           "password" : "${MACHINE_PASSWD}",
           "type" : "PASSWORD"
        }
    }
}
""")

Writing file `shelob-tapis-storage-ysboss.txt'


In [14]:
!systems-addupdate -F ${STORAGE_MACHINE}.txt

Successfully added system shelob-tapis-storage-ysboss


### List files on the STORAGE_MACHINE to ensure access is correctly configured.

In [15]:
#!files-list -S ${STORAGE_MACHINE} ./ | head -5
#!auth-session-init -t tacc.prod -u $AGAVE_USERNAME -p $AGAVE_PASSWD -N $BASE_APP_NAME2
!files-list -l agave://${STORAGE_MACHINE} 

drwx 4096 Oct  8 09:04 ./                 
-rw-  240 Apr 26 06:04 .bash.mike001.sh   
-rw-  181 May  7 03:43 .bash.mike002.sh   
-rw-  236 Nov 15 09:09 .bash.mike003.sh   
-rw-  271 May 11 10:09 .bash.mike004.sh   
-rw-  249 Feb  1 07:07 .bash.mike005.sh   
-rw-  233 Mar  1 06:58 .bash.mike006.sh   
-rw-  271 May  9 17:05 .bash.mike007.sh   
-rw-  271 May  7 06:47 .bash.mike008.sh   
-rw-  240 May  3 08:58 .bash.mike009.sh   
-rw-  271 May  9 14:58 .bash.mike010.sh   
-rw-  271 May  9 18:01 .bash.mike011.sh   
-rw-  271 May  9 16:15 .bash.mike012.sh   
-rw-  271 May  9 17:57 .bash.mike013.sh   
-rw-  271 May  9 16:19 .bash.mike014.sh   
-rw-  242 Mar 22 03:49 .bash.mike015.sh   
-rw-  249 Sep  5 11:37 .bash.mike016.sh   
-rw-  261 Sep  4 11:42 .bash.mike017.sh   
-rw-  237 Jan 22 09:18 .bash.mike018.sh   
-rw-  271 May 11 16:29 .bash.mike019.sh   
-rw-  181 May  7 05:14 .bash.mike020.sh   
-rw-  234 Nov 15 02:51 .bash.mike021.sh   
-rw-  239 Oct  4 11:02 .bash.mike022.sh   
-rw-  235 J

## Step 5: More Machine Details
In order to properly use the execution machine, you will
need to provide agave with details about its queuing system
as well as the default queue that jobs will be run in. Please
edit the next section carefully.

In [16]:
# Configure information about the machine/queue
setvar("""
QUEUE=checkpt
MAX_TIME=72:00:00
NODES=128
PROCS=16
MAX_JOBS=30
""")

os.environ["DIRECTIVES"]=re.sub("\n\\s*",r"\\n","""
#PBS -A ${ALLOCATION}
#PBS -l cput=${MAX_TIME}
#PBS -l walltime=${MAX_TIME}
#PBS -q checkpt
#PBS -l nodes=\${AGAVE_JOB_NODE_COUNT}:ppn=16
""".strip())    
writefile("${EXEC_MACHINE}.txt","""
{
    "id": "${EXEC_MACHINE}",
    "name": "${MACHINE} (${MACHINE_USERNAME})",
    "description": "The ${MACHINE} computer",
    "site": "${DOMAIN}",
    "public": false,
    "status": "UP",
    "type": "EXECUTION",
    "executionType": "HPC",
    "scheduler" : "CUSTOM_TORQUE",
    "environment": null,
    "scratchDir" : "${SCRATCH_DIR}",
    "queues": [
       {
            "customDirectives" : "${DIRECTIVES}",
            "name": "${QUEUE}",
            "default": true,
            "maxJobs": ${MAX_JOBS},
            "maxUserJobs": ${MAX_JOBS},
            "maxNodes": ${NODES},
            "maxProcessorsPerNode": ${PROCS},
            "minProcessorsPerNode": 1,
            "maxRequestedTime": "${MAX_TIME}"
        }
    ],
    "login": {
        "auth": {
         "username" : "${MACHINE_USERNAME}",
         "password" : "${MACHINE_PASSWD}",
         "type" : "PASSWORD"
        },
        "host": "${MACHINE_FULL}",
        "port": ${PORT},
        "protocol": "SSH"
    },
    "maxSystemJobs": 50,
    "maxSystemJobsPerUser": 50,
    "storage": {
        "host": "${MACHINE_FULL}",
        "port": ${PORT},
        "protocol": "SFTP",
        "rootDir": "/",
        "homeDir": "${HOME_DIR}",
        "auth": {
         "username" : "${MACHINE_USERNAME}",
         "password" : "${MACHINE_PASSWD}",
         "type" : "PASSWORD"
        }
     }
    },
    "workDir": "${WORK_DIR}"
}""")                        

!echo ${DIRECTIVES}

!systems-addupdate -F ${EXEC_MACHINE}.txt

QUEUE=checkpt
MAX_TIME=72:00:00
NODES=128
PROCS=16
MAX_JOBS=30
Writing file `shelob-tapis-exec-ysboss.txt'
#PBS -A ${ALLOCATION}
#PBS -l cput=${MAX_TIME}
#PBS -l walltime=${MAX_TIME}
#PBS -q checkpt
#PBS -l nodes=\${AGAVE_JOB_NODE_COUNT}:ppn=16
Successfully added system shelob-tapis-exec-ysboss


### List files on the EXEC_MACHINE to ensure access is correctly configured.

In [17]:
#!files-list -S ${EXEC_MACHINE} ./ | head -5
!files-list -l agave://${EXEC_MACHINE}

drwx 4096 Oct  8 09:04 ./                 
-rw-  240 Apr 26 06:04 .bash.mike001.sh   
-rw-  181 May  7 03:43 .bash.mike002.sh   
-rw-  236 Nov 15 09:09 .bash.mike003.sh   
-rw-  271 May 11 10:09 .bash.mike004.sh   
-rw-  249 Feb  1 07:07 .bash.mike005.sh   
-rw-  233 Mar  1 06:58 .bash.mike006.sh   
-rw-  271 May  9 17:05 .bash.mike007.sh   
-rw-  271 May  7 06:47 .bash.mike008.sh   
-rw-  240 May  3 08:58 .bash.mike009.sh   
-rw-  271 May  9 14:58 .bash.mike010.sh   
-rw-  271 May  9 18:01 .bash.mike011.sh   
-rw-  271 May  9 16:15 .bash.mike012.sh   
-rw-  271 May  9 17:57 .bash.mike013.sh   
-rw-  271 May  9 16:19 .bash.mike014.sh   
-rw-  242 Mar 22 03:49 .bash.mike015.sh   
-rw-  249 Sep  5 11:37 .bash.mike016.sh   
-rw-  261 Sep  4 11:42 .bash.mike017.sh   
-rw-  237 Jan 22 09:18 .bash.mike018.sh   
-rw-  271 May 11 16:29 .bash.mike019.sh   
-rw-  181 May  7 05:14 .bash.mike020.sh   
-rw-  234 Nov 15 02:51 .bash.mike021.sh   
-rw-  239 Oct  4 11:02 .bash.mike022.sh   
-rw-  235 J

### Create the batch script used to run jobs. This should not need editing.

In [18]:
!files-mkdir agave://${EXEC_MACHINE}/${DEPLOYMENT_PATH}

In [19]:
writefile("${BASE_APP_NAME2}-wrapper.txt","""
#!/bin/bash
handle_trap() {
  rc=\$?
  if [ "\$rc" != 0 ]
  then
    \$(\${AGAVE_JOB_CALLBACK_FAILURE})
  fi
}

trap 'handle_trap' ERR EXIT
echo 'running \${simagename} model'
# Setting the x flag will echo every
# command onto stderr. This is
# for debugging, so we can see what's
# going on.
set -x
set -e
echo ==PWD=============
# We also print out the execution
# directory. Again, for debugging purposes.
pwd
echo ==JOB=============

if [ "\${PBS_NODEFILE}" = "" ]
then
 # When running on a system managed by Torque
 # this variable should be set. If it's not,
 # that's a problem.
 echo "The PBS_NODEFILE was not set"
 exit 2
fi

# By default, the PBS_NODEFILE lists nodes multiple
# times, once for each MPI process that should run
# there. We only want one MPI process per node, so
# we create a new file with "sort -u".
LOCAL_NODEFILE=nodefile.txt
sort -u < \${PBS_NODEFILE} > \${LOCAL_NODEFILE}
PROCS=\$(wc -l < \${LOCAL_NODEFILE})

if [ "\${PROCS}" = "" ]
then
 echo "PROCS was not set"
 exit 3
fi

# Prepare the nodes to run the image
export SING_OPTS="--bind \$PWD:/workdir \$SING_OPTS"
for host in \$(cat nodefile.txt)
do
    hostfile="\$HOME/.bash.\${host}.sh"
    echo "export SING_IMAGE=/home/funwave/images/science-base-alt.simg" > \$hostfile
    # echo "export SING_OPTS='\$SING_OPTS'" >> \$hostfile
done

# Create a nodefile that matches our choices at submit time
touch nodes.txt
for i in \$(seq 1 \${AGAVE_JOB_PROCESSORS_PER_NODE})
do
    cat nodefile.txt >> nodes.txt
done

export NP=\$(wc -l < nodes.txt)

tar xzvf input.tgz

mkdir -p output

# /project/singularity/bin/singularity exec \$SING_OPTS /project/sbrandt/chemora/images/\${simagename}.simg bash /usr/local/bin/runapp.sh
/project/singularity/bin/singularity exec \$SING_OPTS /home/funwave/images/science-base-alt.simg bash /usr/local/bin/runapp.sh
mv input/* output/
rm -f output/PRINT*
tar cvzf output.tar.gz output
""")

!files-mkdir agave://${EXEC_MACHINE}/${DEPLOYMENT_PATH}
!files-mkdir agave://${STORAGE_MACHINE}/inputs
!files-cp ${BASE_APP_NAME2}-wrapper.txt agave://${STORAGE_MACHINE}/${DEPLOYMENT_PATH}/

writefile("test.txt","""
parfile="input.txt"
${BASE_APP_NAME2}-wrapper.txt
""")

!files-mkdir agave://${STORAGE_MACHINE}/${DEPLOYMENT_PATH}
!files-cp test.txt agave://${STORAGE_MACHINE}/${DEPLOYMENT_PATH}/

writefile("${BASE_APP_NAME2}.txt","""
{  
    "name":"${BASE_APP_NAME2}",
    "version":"2.0",
    "label":"${BASE_APP_NAME2}",
    "shortDescription":"Run ISAAC app",
    "longDescription":"",
    "deploymentSystem":"${STORAGE_MACHINE}",
    "deploymentPath":"${DEPLOYMENT_PATH}",
    "templatePath":"${BASE_APP_NAME2}-wrapper.txt",
    "testPath":"test.txt",
    "executionSystem":"${EXEC_MACHINE}",
    "executionType":"HPC",
    "parallelism":"PARALLEL",
    "allocation":"${ALLOCATION}",
    "modules":[],
    "inputs":[
        {   
        "id":"input tarball",
        "details":{  
            "label":"input tarball",
            "description":"",
            "argument":null,
            "showArgument":false
        },
        "value":{  
            "default":"",
            "order":0,
            "required":false,
            "validator":"",
            "visible":true
        }
    }   

],
"parameters":[
{
  "id": "simagename",
  "value": {
    "visible": true,
    "required": false,
    "type": "string",
    "order": 0,
    "enquote": false,
    "default": "python",
    "validator": null
  },
  "details": {
    "label": "Singularity Image",
    "description": "The Singularity image to run: swan, funwave",
    "argument": null,
    "showArgument": false,
    "repeatArgument": false
  },
  "semantics": {
    "minCardinality": 0,
    "maxCardinality": 1,
    "ontology": []
  }
}
],
"outputs":[  
    {  
        "id":"Output",
        "details":{  
            "description":"The output",
            "label":"tables"
        },
        "value":{  
            "default":"",
            "validator":""
        }
    }
  ]
}
""")


!apps-addupdate -F ${BASE_APP_NAME2}.txt

setvar("APP_NAME=${BASE_APP_NAME2}-2.0")

print ("Successfully configured Agave")

Writing file `crcollaboratory-tapis-shelob-ysboss-wrapper.txt'
Writing file `test.txt'
Writing file `crcollaboratory-tapis-shelob-ysboss.txt'
Successfully added app crcollaboratory-tapis-shelob-ysboss-2.0
APP_NAME=crcollaboratory-tapis-shelob-ysboss-2.0
Successfully configured Agave


In [20]:
import json, os
from command import cmd
metadata = {"name":os.environ["EXEC_MACHINE"]+"::queue","value":os.environ["QUEUE"]}
c = cmd(["metadata-list","-Q",'{"name":"${EXEC_MACHINE}::queue"}'])
found = False
for k in c["stdout"]:
    ks = k.strip()
    if ks != '':
        cmd(["metadata-addupdate","-F","-",ks],inputs=json.dumps(metadata))
        found = True
        break
if not found:
    cmd(["metadata-addupdate","-F","-"],inputs=json.dumps(metadata))

cmd: metadata-list -Q {"name":"shelob-tapis-exec-ysboss::queue"}
6562886709805126121-242ac11a-0001-012
cmd: metadata-addupdate -F - 6562886709805126121-242ac11a-0001-012
Successfully submitted metadata object 6562886709805126121-242ac11a-0001-012


## Step 6: Grant Access
By default, no one will be able to use the resource you configure unless you specifically grant them access.
The following two functions can be used to grant or revoke access to an Agave user.

In [21]:
# The following two commands enable you to grant or revoke the ability
# to use a given application to a given user
from command import cmd
def grant_user(user):
    cmd("apps-pems-update -u {u} -p READ_EXECUTE $APP_NAME".format(u=user))
    cmd("systems-roles-addupdate -r USER -u {u} $STORAGE_MACHINE".format(u=user))
    cmd("systems-roles-addupdate -r USER -u {u} $EXEC_MACHINE".format(u=user))
    c = cmd(["metadata-list","-Q",'{"name":"${EXEC_MACHINE}::queue"}'])
    for line in c["stdout"]:
        line = line.strip()
        if line != '':
            cmd(["metadata-pems-addupdate","-u",user,"-p","READ",line])
def revoke_user(user):
    cmd("apps-pems-update -u {u} -p NONE $APP_NAME".format(u=user))
    cmd("systems-roles-addupdate -r NONE -u {u} $STORAGE_MACHINE".format(u=user))
    cmd("systems-roles-addupdate -r NONE -u {u} $EXEC_MACHINE".format(u=user))
    c = cmd(["metadata-list","-Q",'{"name":"${EXEC_MACHINE}::queue"}'])
    for line in c["stdout"]:
        line = line.strip()
        if line != '':
            cmd(["metadata-pems-addupdate","-u",user,"-p","NONE",line])

In [22]:
!auth-tokens-refresh

Refreshing access token...
129abda9aeb7b9e49a66fc899187a37
New token expires: Tue Oct 8 17:14:15 CDT 2019
No handlers could be found for logger "agavepy.utils.save_configs"


In [23]:
!auth-session-init -t tacc.prod -u $AGAVE_USERNAME -p $AGAVE_PASSWD

Loading None for ysboss from tacc.prod
Refreshing access token for crcollaboratory-tapis-shelob-ysboss...
129abda9aeb7b9e49a66fc899187a37
No handlers could be found for logger "agavepy.utils.save_configs"


In [24]:
grant_user("tg457049")

cmd: apps-pems-update -u tg457049 -p READ_EXECUTE crcollaboratory-tapis-shelob-ysboss-2.0
Successfully updated permission for tg457049
cmd: systems-roles-addupdate -r USER -u tg457049 shelob-tapis-storage-ysboss
Successfully updated roles for user tg457049 on shelob-tapis-storage-ysboss
cmd: systems-roles-addupdate -r USER -u tg457049 shelob-tapis-exec-ysboss
Successfully updated roles for user tg457049 on shelob-tapis-exec-ysboss
cmd: metadata-list -Q {"name":"shelob-tapis-exec-ysboss::queue"}
6562886709805126121-242ac11a-0001-012
cmd: metadata-pems-addupdate -u tg457049 -p READ 6562886709805126121-242ac11a-0001-012
Successfully updated permission for tg457049
